In [ ]:
from azure.cosmos import CosmosClient
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey
import azure.cosmos.exceptions as exceptions
from dotenv import load_dotenv
from openai import AzureOpenAI
import json

load_dotenv(verbose=True)

In [2]:
def query_items(container, doc_id):
    print('\n1.4 Querying for an  Item by Id\n')

    # enable_cross_partition_query should be set to True as the container is partitioned
    #a = container.query_items(
    #    query="SELECT * FROM c WHERE c.id=@id",
    #    parameters=[
    #        {"name": "@id", "value": doc_id}
    #    ],
    #    enable_cross_partition_query=True
    #)

    #items = list(container.query_items(
    #    query="SELECT * FROM c",
    #    enable_cross_partition_query=True
    #))
    a = container.query_items(
        query="SELECT * FROM c",
        enable_cross_partition_query=True
    )
    
    import pandas as pd
    df = pd.read_json(a)
    return df
    #obj = json.dumps(array)
    #p = json.load(obj)
    #print(obj)
    #print('Item queried by Id {0}'.format(items[0].get("id")))


In [3]:
import os
url = os.environ['ACCOUNT_URI']
tenant_id = os.environ['TENANT_ID']
#client_id = os.environ['CLIENT_ID']
#client_secret = os.environ['CLIENT_SECRET']
database_id = os.environ['DATABASE_ID']
container_id = os.environ['CONTAINER_ID']
# Using ClientSecretCredential
#aad_credentials = ClientSecretCredential(
#    tenant_id=tenant_id,
#    client_id=client_id,
#    client_secret=client_secret)
#
# Using DefaultAzureCredential (recommended)
#https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/cosmos/azure-cosmos/samples/document_management.py
aad_credentials = DefaultAzureCredential()

cosmosdb_client = CosmosClient(url, aad_credentials)

In [4]:
aoaiclient = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
model = os.environ['AZURE_OPENAI_MODEL_EMBEDDING']

In [ ]:
text = "昨日は暑かった。"
embedding_value = aoaiclient.embeddings.create(input = [text], model=model).data[0].embedding
print(embedding_value)
df = None

In [ ]:
try:
    # setup database for this sample
    db = cosmosdb_client.create_database_if_not_exists(id=database_id)
    # setup container for this sample
    container = db.create_container_if_not_exists(id=container_id,
                                                  partition_key=PartitionKey(path='/id', kind='Hash'))
    df = query_items(container=container, doc_id="aa6999d2-3aa3-4f52-b413-f1c550f806a2")
    print(df)
except exceptions.CosmosHttpResponseError as e:
    print('\nrun_sample has caught an error. {0}'.format(e.message))

In [ ]:
print(df)